In [2]:
! pip install docplex

    100% |████████████████████████████████| 532kB 861kB/s ta 0:00:01
  Running setup.py bdist_wheel for docplex ... done
  Stored in directory: /Users/eduardopetecofmattoso/Library/Caches/pip/wheels/5d/27/dd/9ee8195bca0c628f6356cf57bc9e7dbd60b80265b74203d329
  Running setup.py bdist_wheel for docloud ... done
  Stored in directory: /Users/eduardopetecofmattoso/Library/Caches/pip/wheels/4e/24/19/9b0e57545d7543ecec35c5faff78c058bf0c2506369c8b1d31
Successfully built docplex docloud


In [3]:
from docplex.mp.model import Model
from docplex.util.environment import get_environment
import numpy as np
import pandas as pd

In [17]:
url = "https://api-oaas.docloud.ibmcloud.com/job_manager/rest/v1/"
key = "<api_key>"

In [5]:
myInput =[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [6]:
R = range(1, 10)
centers = [(2,2), (2,5), (2,8), (5,2), (5,5), (5,8), (8,2), (8,5), (8,8)]
radius = [-1, 0, 1]
idx = [(i, j, k) for i in R for j in R for k in R]

In [7]:
model = Model("sudoku")
x = model.binary_var_dict(idx, name="board")

In [8]:
for i in R:
    for j in R:
        if myInput[i - 1][j - 1] != 0:
            model.add_constraint(x[i, j, myInput[i - 1][j - 1]] == 1)

In [9]:
for i in R:
    for j in R:
        model.add_constraint(model.sum(x[i, j, k] for k in R) == 1)

In [10]:
for j in R:
    for k in R:
        model.add_constraint(model.sum(x[i, j, k] for i in R) == 1)

In [11]:
for i in R:
    for k in R:
        model.add_constraint(model.sum(x[i, j, k] for j in R) == 1)

In [12]:
for c in centers:
    for k in R:
        model.add_constraint(model.sum(x[c[0]+r1, c[1]+r2, k] for r1 in radius for r2 in radius) == 1)

In [13]:
model.minimize(model.sum(x[c[0],c[1],k]*k for k in R for c in centers))

In [14]:
solution = model.solve(url=url, key=key, log_output=True)

[2019-08-05T13:30:23Z, INFO] CPLEX version 12090000
[2019-08-05T13:30:23Z, INFO] Parameter file:
[2019-08-05T13:30:23Z, INFO] # -- This content is generated by DOcplex
[2019-08-05T13:30:23Z, INFO] CPLEX Parameter File Version 12.8.0.0
[2019-08-05T13:30:23Z, INFO] # --- end of generated prm data ---
[2019-08-05T13:30:23Z, WARN] Warning:  CPLEX version found in file python_sudoku.prm (12.8.0.0) does not match
[2019-08-05T13:30:23Z, WARN]           this CPLEX version (12.9.0.0).
[2019-08-05T13:30:23Z, WARN] Changed parameter CPX_PARAM_THREADS from 0 to 10
[2019-08-05T13:30:23Z, INFO] Param[1,067] = 10
[2019-08-05T13:30:23Z, INFO] Param[1,130] = utf-8
[2019-08-05T13:30:23Z, INFO] Param[1,132] = -1
[2019-08-05T13:30:23Z, INFO] CPXPARAM_Threads                                 10
[2019-08-05T13:30:23Z, INFO] CPXPARAM_Output_CloneLog                         -1
[2019-08-05T13:30:23Z, INFO] CPXPARAM_Read_APIEncoding                        "utf-8"
[2019-08-05T13:30:23Z, INFO] Tried aggregator 1 t

In [16]:
solution_values = solution.get_all_values()
sudoku_solution = np.zeros((9,9))

for i,v in enumerate(x):
    if(solution_values[i] == 1):
        sudoku_solution[v[0]-1][v[1]-1] = v[2]
        
print(sudoku_solution)

[[8. 6. 1. 9. 7. 2. 5. 4. 3.]
 [9. 2. 4. 8. 3. 5. 7. 1. 6.]
 [7. 5. 3. 1. 4. 6. 8. 9. 2.]
 [1. 4. 8. 2. 5. 3. 9. 6. 7.]
 [5. 3. 7. 6. 1. 9. 4. 2. 8.]
 [2. 9. 6. 4. 8. 7. 3. 5. 1.]
 [3. 8. 2. 5. 6. 4. 1. 7. 9.]
 [4. 1. 9. 7. 2. 8. 6. 3. 5.]
 [6. 7. 5. 3. 9. 1. 2. 8. 4.]]
